https://arxiv.org/abs/2007.14044

In [2]:
!pip3 install qiskit --upgrade

In [3]:
import qiskit
from scipy.special import erfinv
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.model_selection import train_test_split

qs = qiskit.Aer.get_backend('qasm_simulator')

## IRIS Dataset

In [4]:
diconvert = {
    "Iris-setosa" : "00",
    "Iris-versicolor" : "01",
    "Iris-virginica" : "10",
    "NaN" : "11"
}

dicinv = {diconvert[k] : k for k in diconvert} 

## Dataset preparation

In [5]:
def qFactor(eps, d):
    return erfinv(1-eps**(1/d)/2)

In [6]:
eps = 0.01
alpha = 0.1
d = 4
q = qFactor(eps, d)

In [7]:
q

0.9980412728799112

In [8]:
def standardise(x):
    return (x-np.mean(x))/np.std(x)

def rescaleFeature(x):
    return (1-alpha/2)*(np.pi/q)*standardise(x)

In [9]:
df = pd.read_csv("iris.data")
attributes = df.columns[:-1]
for x in attributes:
    df[x] = rescaleFeature(df[x])

In [ ]:
def psiGate(qc, param, bit):
    qc.rx(np.pi/2, bit)
    qc.rz(param, bit)
    qc.rx(np.pi/2, bit)

In [10]:
def irisCircuit(theta, omega):
    qc = qiskit.QuantumCircuit(2)
    for i in range(4):
        if i : qc.cz(0, 1)
        psiGate(qc, omega[(2*i) % 4], 0)
        psiGate(qc, omega[(2*i+1) % 4], 1)
        qc.cz(0, 1)
        psiGate(qc, omega[(2*i)], 0)
        psiGate(qc, omega[(2*i+1)], 1)
    return qc

In [11]:
def predictionDict(theta, omega):
    
    qc = qiskit.QuantumCircuit(2, 2)
    qc.append(irisCircuit(theta, omega), range(2))
    qc.measure(range(2), range(2))
    
    job = qiskit.execute(qc, shots=nShots, backend=qs)
    c = job.result().get_counts()
    
    return c


In [12]:
def lossFunction(theta, data):
    omega = data[attributes].values
    label = diconvert[data["class"]]
    c = predictionDict(theta, omega)
    
    if label in c:
        e = np.exp(c[label]/nShots)
    else :
        e = 1
    s = np.exp(np.array(list(c.values()))/nShots).sum()
    return -np.log(e/s)


In [13]:
def lossBatch(theta, batch):
    s = batch.apply(
        lambda data : lossFunction(theta, data),
        axis=1
    )
    return s.mean()                

In [14]:
train_size = 90
test_size = 60

In [15]:
trainSet, testSet = train_test_split(
    df, test_size=test_size, train_size=train_size, stratify = df[["class"]])

testSet = pd.DataFrame(testSet)
trainSet= pd.DataFrame(trainSet)

In [16]:
theta_init = np.random.uniform(-np.pi, np.pi, 8)

In [17]:
nShots = 250

In [18]:
opt = sp.optimize.minimize(
    fun=lambda theta : lossBatch(theta, trainSet),
    x0 = new_t,
    method='COBYLA',
    options={'maxiter': 20}
)
new_t = opt.x

NameError: name 'new_t' is not defined

In [ ]:
opt

     fun: 1.1777512427472934
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 20
  status: 2
 success: False
       x: array([-0.80097474, -1.83879715, -2.10321306, -1.94008108,  3.58145318,
       -4.19574068, -0.84104109,  2.2175159 ])

In [ ]:
nShots = 500

In [ ]:
opt = sp.optimize.minimize(
    fun=lambda theta : lossBatch(theta, trainSet),
    x0 = new_t,
    method='COBYLA',
    options={'maxiter': 30}
)
new_t = opt.x

In [ ]:
opt

     fun: 1.1714276240079242
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 30
  status: 2
 success: False
       x: array([-0.73231423, -1.69128226, -1.88221496, -1.97487909,  3.56065974,
       -4.36596299, -0.71698384,  2.40100239])

In [ ]:
opt = sp.optimize.minimize(
    fun=lambda theta : lossBatch(theta, trainSet),
    x0 = new_t,
    method='COBYLA',
    options={'maxiter': 70}
)

In [ ]:
theta_opt = opt.x

In [ ]:
opt

     fun: 1.1581348977374308
   maxcv: 0.0
 message: 'Maximum number of function evaluations has been exceeded.'
    nfev: 70
  status: 2
 success: False
       x: array([-0.76985744, -1.66052885, -1.9165268 , -1.69680548,  3.63944231,
       -4.45487365, -0.74240939,  2.37330525])

In [ ]:
def argmaxDict(c):
    v = None
    for key in c :
        if v is None or c[key] >= v:
            k = key
            v = c[key]
    return k

In [ ]:
def prediction(theta, omega):
    return argmaxDict(predictionDict(theta, omega))

In [ ]:
def predictedClass(data):
    omega = data[attributes].values.T
    ypred = prediction(theta_opt, omega)
    return dicinv[ypred]

In [ ]:
nShots = 1800

In [ ]:
testSet["predicted"] = testSet.apply(
    lambda row : dicinv[prediction(theta_opt, row[attributes].values)],
    axis=1
)

In [ ]:
sum(np.array(testSet["class"] == testSet["predicted"]))/60

0.45

In [ ]:
trainSet["predicted"] = trainSet.apply(
    lambda row : dicinv[prediction(theta_opt, row[attributes].values)],
    axis=1
)

In [ ]:
TOKEN = ...

In [ ]:
qiskit.IBMQ.save_account(TOKEN, overwrite=True) 
provider = qiskit.IBMQ.load_account()
backend = qiskit.providers.ibmq.least_busy(provider.backends())

mapped_circuit = qiskit.transpile(circuit, backend=backend)
qobj = qiskit.assemble(mapped_circuit, backend=backend, shots=1024)

IBMQAccountCredentialsInvalidToken: 'Invalid IBM Quantum Experience token found: "Ellipsis" of type <class \'ellipsis\'>.'

In [ ]:
# execute the circuit
job = backend.run(qobj)


In [ ]:
job.status()


In [ ]:
ID = job.job_id()

In [ ]:
job = backend.retrieve_job(ID)

In [ ]:
result = job.result()

In [ ]:
counts = result.get_counts()

In [ ]:
counts

# Gaussian XOR

In [ ]:
def xorCircuit(theta, omega):
    qc = qiskit.QuantumCircuit(2)
    psiGate(qc, omega[0], 0)
    psiGate(qc, omega[1], 1)
    for i in range(2):
        qc.cz(0, 1)
        psiGate(qc, theta[2*i], 0)
        psiGate(qc, theta[2*i+1], 1)
    return qc

# Skin segmentation

In [ ]:
def skinSegCircuit(theta, omega):
    qc = qiskit.QuantumCircuit(3)
    psiGate(qc, omega[0], 0)
    psiGate(qc, omega[1], 1)
    psiGate(qc, omega[2], 2)
    
    qc.cz(0, 2)
    
    psiGate(qc, theta[0], 0)
    psiGate(qc, theta[1], 2)
    
    qc.cz(0, 1)
    
    psiGate(qc, theta[2], 0)
    psiGate(qc, theta[3], 1)
    
    qc.cz(1, 2)
    
    psiGate(qc, theta[4], 1)
    psiGate(qc, theta[5], 2)
    return qc

# Synthetic dataset with 4 classes

In [ ]:
def syntheticCircuit(theta, omega):
    qc = qiskit.QuantumCircuit(2)
    for i in range(2):
        if i : qc.cz(0, 1)
        psiGate(qc, omega[0], 0)
        psiGate(qc, omega[1], 1)
        qc.cz(0, 1)
        psiGate(qc, omega[(6*i)], 0)
        psiGate(qc, omega[(6*i+1)], 1)
        qc.cz(0, 1)
        psiGate(qc, omega[0], 0)
        psiGate(qc, omega[1], 1)
        qc.cz(0, 1)
        psiGate(qc, omega[(6*i+2)], 0)
        psiGate(qc, omega[(6*i+3)], 1)
        qc.cz(0, 1)
        psiGate(qc, omega[(6*i+4)], 0)
        psiGate(qc, omega[(6*i+5)], 1)
    return qc